# Import packages

In [1]:
import theano
import theano.tensor as T
import numpy as np
import h5py
floatX = theano.config.floatX

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


# Load data to VRAM

In [2]:
h5f = h5py.File('mnist.hdf5','r')
data_x = theano.shared(h5f['train_data'][()])
data_y = theano.shared(np.asarray(h5f['train_label'][()], dtype='int32'))

# Create parameters in VRAM

In [3]:
w1 = theano.shared(np.ones((784, 300),dtype=floatX),name='w1')
w2 = theano.shared(np.ones((300, 10), dtype=floatX),name='w2')
b1 = theano.shared(np.zeros((300,), dtype=floatX),name='b1')
b2 = theano.shared(np.zeros((10,), dtype=floatX),name='b2')
params = [w1,w2,b1,b2]

# Model definition

$h_1=\sigma(w_1 x+b_1)$

$y=\sigma(w_2 h_1+b_2)$

$cost = -\frac{1}{N}\sum\limits_{i=1}^N \log y_i^{t_i} $ (Cross Entropy)

In [4]:
x = T.matrix('x')
t = T.ivector('t')
h1 = T.nnet.sigmoid(T.dot(x,w1)+b1)
y = T.nnet.softmax(T.dot(h1,w2)+b2)
cost = -T.mean(T.log(y)[T.arange(t.shape[0]), t])

# Compile function

In [5]:
i = T.iscalar('i')
batch_size = 200
n_batches = int(data_x.get_value().shape[0] / batch_size)

grads = T.grad(cost, params)
nu = 0.01
updates = [(p, p-nu*grad) for p,grad in zip(params, grads)]
train_model = theano.function([i], cost, updates=updates, 
                              givens=[(x,data_x[i*batch_size:(i+1)*batch_size]),
                                      (t,data_y[i*batch_size:(i+1)*batch_size])])

# Training

In [6]:
for epoch in xrange(100):
    cost_vals = []
    for bi in xrange(n_batches):
        cost_val = train_model(bi)
        cost_vals.append(cost_val)
    print("Epoch {} Cost {}".format(epoch, np.mean(cost_vals)))

Epoch 0 Cost 2.30424356461
Epoch 1 Cost 2.30423760414
Epoch 2 Cost 2.30423736572
Epoch 3 Cost 2.30423688889
Epoch 4 Cost 2.30423760414
Epoch 5 Cost 2.30423688889
Epoch 6 Cost 2.30423855782
Epoch 7 Cost 2.30423760414
Epoch 8 Cost 2.30423593521
Epoch 9 Cost 2.30423855782
Epoch 10 Cost 2.30423736572
Epoch 11 Cost 2.30423688889
Epoch 12 Cost 2.30423927307
Epoch 13 Cost 2.30423688889
Epoch 14 Cost 2.30423736572
Epoch 15 Cost 2.30423545837
Epoch 16 Cost 2.30423736572
Epoch 17 Cost 2.30423641205
Epoch 18 Cost 2.30423784256
Epoch 19 Cost 2.30423593521
Epoch 20 Cost 2.30423736572
Epoch 21 Cost 2.30423784256
Epoch 22 Cost 2.30423927307
Epoch 23 Cost 2.3042383194
Epoch 24 Cost 2.30423736572
Epoch 25 Cost 2.30423688889
Epoch 26 Cost 2.30423855782
Epoch 27 Cost 2.30423641205
Epoch 28 Cost 2.30423784256
Epoch 29 Cost 2.30423784256
Epoch 30 Cost 2.30423808098
Epoch 31 Cost 2.3042383194
Epoch 32 Cost 2.3042371273
Epoch 33 Cost 2.3042371273
Epoch 34 Cost 2.30423736572
Epoch 35 Cost 2.30423760414
Epoch 